linear层，现在都不带bias

In [ ]:
from jaxtyping import Float, Int
# JAXTyping 是一个为 JAX、PyTorch 等深度学习框架提供的类型注解库，用于标注张量的形状和数据类型。
def run_linear(
    d_in: int,
    d_out: int,
    weights: Float[Tensor, " d_out d_in"],
    in_features: Float[Tensor, " ... d_in"],
) -> Float[Tensor, " ... d_out"]:

线性层主要的作用是改变特征维度，必须可微，最简单的就是投影了
会输入 d_in 和 d_out 作为输入特征和输出特征的维度，
weights 作为权重矩阵，in_features 作为输入特征张量。

weights的数据类型是Float（浮点数），张量类型是torch的Tensor，形状为 "d_out d_in"，表示输出特征维度和输入特征维度的矩阵。
in_features " ... d_out" 表示输入特征张量的形状，其中 "..." 表示可以有任意数量的前置维度。
比如可能是 batch ，seq_len, d_in

那么很简单，[seq_len, d_in] 矩阵乘[d_in, d_out] 矩阵
就可以了
拿in_features的后两维，与weights的转置进行矩阵乘法

参数:
        d_in (int): 输入维度的大小
        d_out (int): 输出维度的大小
        weights (Float[Tensor, "d_out d_in"]): 要使用的线性权重
        in_features (Float[Tensor, "... d_in"]): 要应用函数的输出张量

    返回:
        Float[Tensor, "... d_out"]: 线性模块的变换输出。
    

In [ ]:
class Linear(nn.Module):
    def __init__(self,in_features,out_features,device=None,dtype=None):
        super().__init__()
            self.in_features = in_features
            self.out_features = out_features
            # 创建权重参数矩阵 W: (out_features, in_features)
            # 注意：存储为W而不是W的转置，便于内存访问
            self.weight = nn.Parameter(
                torch.empty(
                    out_features,
                    in_features,
                    device=device,
                    dtype=dtype
                )
            )
            # 初始化权重
            self._reset_parameters()

        def _reset_parameters(self): 
            """
            初始化空的矩阵
            """
            # 使用截断正态分布初始化权重
            # std = sqrt(2 / (in_features + out_features)) 是常用的Xavier初始化变体
            std = (2.0 / (self.in_features + self.out_features)) ** 0.5
            torch.nn.init.trunc_normal_(self.weight, std=std)

        def forward(self,x: Tensor)-> Tensor:
            # 矩阵乘法有广播机制，只会对最后两维进行矩阵乘法
            return x @ self.weight.T

In [ ]:
import einx
# 可以使用einx库来简化矩阵乘法的操作

def forward(self, x: Tensor) -> Tensor:
        # 使用 einx 进行矩阵乘法
        # "... d_in, d_out d_in -> ... d_out" 表示：
        # 输入: (..., d_in)，权重: (d_out, d_in)
        # 输出: (..., d_out)
        return einx.dot("... d_in, d_out d_in -> ... d_out", x, self.weight)


In [ ]:
# 传统方式 重塑操作
x = x.view(batch_size, seq_len, num_heads, d_head)

# einx 方式
x = einx.rearrange("batch seq (heads d_head) -> batch seq heads d_head", 
                   x, heads=num_heads)


# 沿着序列长度求和
# 传统方式
result = x.sum(dim=1)  # (batch, seq, d_model) -> (batch, d_model)

# einx 方式
result = einx.sum("batch [seq] d_model -> batch d_model", x) 





# einx 平均
result = einx.mean("batch [seq] d_model -> batch d_model", x)